# 2

In [1]:
from collections import Counter
import matplotlib.pyplot as plt
import json
import copy

In [34]:
data = []
with open('raw/all_pairs.txt', 'r', encoding='utf-8') as file:
    file.readline()     # 读掉第一行
    for line in file:
        data.append(eval(line))
data[:5]

[('/api/google-maps', '/api/twitter', 0, 151, 184, 0.82),
 ('/api/twitter', '/api/google-maps', 0, 151, 184, 0.82),
 ('/api/flickr', '/api/youtube', 0, 139, 184, 0.76),
 ('/api/youtube', '/api/flickr', 0, 139, 184, 0.76),
 ('/api/facebook', '/api/twitter', 0, 159, 180, 0.88)]

In [7]:
def remove_symmetry(data):
    # 清理数据，把对称的删除
    data2 = copy.deepcopy(data)
    for d in data2:
        rev_d = (d[1], d[0], d[2], d[3], d[4], d[5])
        if rev_d in data2:
            data2.remove(d)
    return data2

[('/api/twitter', '/api/google-maps', 0, 62, 184, 0.0),
 ('/api/flickr', '/api/youtube', 0, 36, 184, 0.0),
 ('/api/twitter', '/api/facebook', 0, 64, 179, 0.0),
 ('/api/flickr', '/api/google-maps', 0, 46, 171, 0.0),
 ('/api/google-maps', '/api/youtube', 0, 48, 157, 0.0),
 ('/api/twilio-sms', '/api/twilio', 0, 42, 152, 0.0)]

读取原始数据，然后获取我们实际访问出来的Mashup存活情况

In [16]:
with open('raw/api_mashup/active_mashups_data.txt', 'r', encoding='utf-8') as act_file:
    act_mashups = json.load(act_file)
    act_mashups = [m for m in act_mashups if m is not None]

with open('raw/api_mashup/deadpool_mashups_data.txt', 'r', encoding='utf-8') as dead_file:
    dead_mashups = json.load(dead_file)
    dead_mashups = [m for m in dead_mashups if m is not None]

all_mashups = act_mashups + dead_mashups

In [17]:
with open('raw/accessiblility/api_accessbility/mashup_accessibility.txt', 'r') as acc:
    accessibility = json.load(acc)
    # accessibility = [ac for ac in accessibility if ac is not None]
    # build a dict
    acc_dict = {}
    for ac in accessibility:
        if ac is None:
            continue
        else:
            if ac['is_accessible']:
                acc_dict[ac['title']] = True
            else:
                acc_dict[ac['title']] = False

def is_accessible(title):
    return acc_dict.get(title, False)

def produce_pair(apis, requires_reflexive=False, requires_symmetry=False):
    pairs = set()
    sorted_apis = sorted(apis, key=lambda x: x[0])
    for l in sorted_apis:
        if requires_reflexive:
            pairs.add((l, l, 0))
        for r in sorted_apis:
            if l != r:
                if requires_symmetry:
                    pairs.add((l, r, 0))
                else:
                    # 不要对称的
                    if (r, l, 0) not in pairs:
                        pairs.add((l, r, 0))
    return list(pairs)

### 制作数据结构
1. mashup_title -> [pair] (list)
2. pair -> [mashup_title]
3. mashup_title -> mashup_object

In [18]:
api_pairs = {}    # mashup和他的pair
api_pair_mashups = {}
mashups_name_dict = {}
for dd in all_mashups:
    mashups_name_dict[dd['title']] = dd
    ra_links = []
    if dd is None:
        continue
    for ra in dd['related_apis']:
        if ra is None:
            continue
        ra_links.append(ra['url'])
    pairs = produce_pair(ra_links, False, True)
    api_pairs[dd['title']] = pairs
    for pair in pairs:
        if pair in api_pair_mashups:
            api_pair_mashups[pair].append(dd['title'])
        else:
            api_pair_mashups[pair] = [dd['title']]

len(api_pairs), len(api_pair_mashups)


(7901, 29078)

In [19]:
api_pairs_counter_all = Counter()
api_pairs_counter_act = Counter()
for mashup_name in api_pairs:
    pairs = api_pairs[mashup_name]
    for pair in pairs:
        api_pairs_counter_all[pair] += 1
        if is_accessible(mashup_name):
            api_pairs_counter_act[pair] += 1

In [20]:
# 整理数据
api_pairs_list = []
for pair in api_pairs_counter_all.keys():
    if pair in api_pairs_counter_act:
        new_pair = (pair[0], pair[1], pair[2], api_pairs_counter_act[pair], api_pairs_counter_all[pair], 0.)
    else:
        # 不在act字典，说明没有被活着的mashup引用
        new_pair = (pair[0], pair[1], pair[2], 0, api_pairs_counter_all[pair], 0.)
    api_pairs_list.append(new_pair)
api_pairs_list = sorted(api_pairs_list, key=lambda x: x[4], reverse=True)
api_pairs_list = remove_symmetry(api_pairs_list)

In [21]:
api_pairs_list[:10]

[('/api/google-maps', '/api/twitter', 0, 62, 184, 0.0),
 ('/api/twitter', '/api/google-maps', 0, 62, 184, 0.0),
 ('/api/flickr', '/api/youtube', 0, 36, 184, 0.0),
 ('/api/youtube', '/api/flickr', 0, 36, 184, 0.0),
 ('/api/facebook', '/api/twitter', 0, 64, 179, 0.0),
 ('/api/twitter', '/api/facebook', 0, 64, 179, 0.0),
 ('/api/flickr', '/api/google-maps', 0, 46, 171, 0.0),
 ('/api/google-maps', '/api/flickr', 0, 46, 171, 0.0),
 ('/api/youtube', '/api/google-maps', 0, 48, 157, 0.0),
 ('/api/google-maps', '/api/youtube', 0, 48, 157, 0.0)]

In [36]:
act_count_pair = 0
dead_count_pair = 0
act_count_mashup = 0
dead_count_mashup = 0

for dd in all_mashups:
    ra_links = []
    if dd is None:
        continue
    for ra in dd['related_apis']:
        if ra is None:
            continue
        ra_links.append(ra['url'])
    temp_pairs = produce_pair(ra_links, False, True)
    if is_accessible(dd['title']):
        act_count_mashup += 1
    else:
        dead_count_mashup += 1
    for pair in temp_pairs:
        count = api_pairs_counter_all[pair]
        if is_accessible(dd['title']):
            act_count_pair += count
        else:
            dead_count_pair += count

act_count_pair / float(act_count_mashup), dead_count_pair / float(dead_count_mashup)




(99.856, 112.93739673214614)

## 统计Mashup中api的被使用次数

In [8]:
api_usage_counter = Counter()
for ma in all_mashups:
    ras = ma['related_apis']
    for ra in ras:
        if ra is None:
            continue
        api_usage_counter[ra['url']] += 1

len(api_usage_counter)

1693

In [9]:
list(sorted(api_usage_counter.items(), key=lambda x: x[1], reverse=True))[:10]

[('/api/google-maps', 2575),
 ('/api/twitter', 827),
 ('/api/youtube', 710),
 ('/api/flickr', 635),
 ('/api/facebook', 451),
 ('/api/amazon-product-advertising', 433),
 ('/api/twilio', 361),
 ('/api/lastfm', 245),
 ('/api/ebay', 227),
 ('/api/google-search', 194)]

In [10]:
def get_usage(url):
    return api_usage_counter[url]

In [11]:
# 获得api在Mashup中的使用次数
get_usage('/api/google-maps')

2575

已经有了pair->mashups

还需要mashup->pairs

In [12]:
mashup_count_pairs = {}
for pl in data:
    temp_pair = (pl[0], pl[1], pl[2])
    for mashup in api_pair_mashups[temp_pair]:
        if mashup in mashup_count_pairs:
            mashup_count_pairs[mashup].append(pl)
        else:
            mashup_count_pairs[mashup] = [pl]

list(mashup_count_pairs.items())[:3]


[('Mashup: PropRover',
  [('/api/twitter', '/api/google-maps', 0, 62, 184, 0.0),
   ('/api/feedburner', '/api/twitter', 0, 2, 7, 0.0),
   ('/api/feedburner', '/api/google-maps', 0, 2, 6, 0.0),
   ('/api/feedburner', '/api/facebook-business-mapping', 0, 0, 1, 0.0),
   ('/api/facebook-business-mapping', '/api/twitter', 0, 0, 1, 0.0),
   ('/api/feedburner', '/api/google-my-business', 0, 0, 1, 0.0),
   ('/api/google-my-business', '/api/facebook-business-mapping', 0, 0, 1, 0.0),
   ('/api/google-maps', '/api/facebook-business-mapping', 0, 0, 1, 0.0),
   ('/api/twitter', '/api/google-my-business', 0, 0, 1, 0.0),
   ('/api/facebook-business-mapping', '/api/feedburner', 0, 0, 1, 0.0),
   ('/api/google-maps', '/api/google-my-business', 0, 0, 1, 0.0)]),
 ('Mashup: We Read, We Tweet',
  [('/api/twitter', '/api/google-maps', 0, 62, 184, 0.0),
   ('/api/google-maps', '/api/new-york-times-article-search', 0, 0, 4, 0.0),
   ('/api/twitter', '/api/backtweets', 0, 0, 3, 0.0),
   ('/api/new-york-times-a